In [7]:
import numpy as np
import math
import nest
import nest.raster_plot
import matplotlib.pyplot as plt
import scipy.stats as stats

In [9]:
def simulate(itds, ilds, MNTBCs2MSO_weights, sim_time, rate_group3, rate_group4, rate_group5):

    n_battery = len(MNTBCs2MSO_weights)
    
    for num_itd in range(len(itds)):

        itd = itds[num_itd]
        ild = ilds[num_itd]
        print(itd)
        print(ild)
        
        nest.ResetKernel()
        nest.local_num_threads = 16
        nest.resolution = 0.01 # 10 us = minimum audible angle (MAA) increment of 1.25°


        tone = 100
        tone_error = 2

        #angle = 0
        max_ild = 2
        mean_amplitude = 1000

        ANFs2SBCs_weight = 2.0
        ANFs2GBCs_weight = 1.0
        SBCs2MSO_weight = 4.0
        #MNTBCs2MSO_weights = [-2.0, -4.0, -6.0, -8.0, -10.0, -12.0, -14.0, -16.0, -20.0, -24.0]
        GBCs2MNTBCs_weight = 16.0

        Cm_bushy = 10
        delay = 0.1
        
        noise_rate = 20

        n_ANFs = 35000
        mean_rate = 0
        ANFs2SBCs = 4
        ANFs2GBCs = 20
        n_SBCs = int(n_ANFs/ANFs2SBCs)
        n_GBCs = int(n_ANFs/ANFs2GBCs)

        freq = np.round(np.logspace(np.log(20),np.log(20000),num = 3500, base = np.exp(1)),2)

        spectro = np.zeros((3500,sim_time))
        channels = len(freq[np.where((freq>tone-tone_error) & (freq<tone+tone_error))])
        amplitudes = stats.norm.pdf(np.linspace(-1, 1, channels) , 0, 1.0/(math.sqrt(2*math.pi)*1))
        spectro[np.where(freq>tone-tone_error)[0][0]:np.where((freq<tone+tone_error))[0][-1]+1, :] = amplitudes.reshape(channels,1)*np.ones((channels, sim_time))

        #angle = 0
        #w_head = 22 #cm
        #v_sound = 33000 #cm/s
        #delta_x = (w_head*np.sin(np.deg2rad(angle)))
        #itd = 1000*delta_x/v_sound #ms
        ipds = 2*np.pi*itd*freq/1000
        #ild = angle*max_ild/90

        # Populations

        r_ANFs_amp = nest.Create('sinusoidal_poisson_generator',n_ANFs,
                    params={'frequency': np.repeat(freq, 10),
                            'phase': np.repeat(np.rad2deg(ipds),10)}) #ITDs

        r_ANFs = nest.Create('parrot_neuron',n_ANFs)

        r_SBCs = nest.Create('iaf_cond_alpha', n_SBCs, 
                         params = {'C_m': Cm_bushy, 'V_m': -70})

        r_GBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': Cm_bushy, 'V_m': -70})

        r_MNTBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': Cm_bushy, 'V_m': -70})

        l_MSO = nest.Create('iaf_cond_alpha', n_GBCs*n_battery, 
                         params = {'C_m': Cm_bushy, 'V_m': -70})

        l_ANFs_amp = nest.Create('sinusoidal_poisson_generator',n_ANFs,
                    params={'frequency': np.repeat(freq, 10),
                        'phase': 0})

        l_ANFs = nest.Create('parrot_neuron',n_ANFs)

        l_SBCs = nest.Create('iaf_cond_alpha', n_SBCs, 
                         params = {'C_m': Cm_bushy, 'V_m': -70})

        l_GBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': Cm_bushy, 'V_m': -70})

        l_MNTBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': Cm_bushy, 'V_m': -70})

        #l_MSO = nest.Create('iaf_cond_alpha', n_GBCs, 
         #                params = {'C_m': 23, 'V_m': -70})

        ANFs_noise = nest.Create('poisson_generator',1,
                         params = {'rate':noise_rate})

        s_rec_r = nest.Create('spike_recorder')
        s_rec_l = nest.Create('spike_recorder')

        nest.Connect(r_ANFs_amp, r_ANFs, 'one_to_one')
        nest.Connect(l_ANFs_amp, l_ANFs, 'one_to_one')

        nest.Connect(r_ANFs, s_rec_r, 'all_to_all')
        nest.Connect(l_ANFs, s_rec_l, 'all_to_all')

        nest.Connect(r_SBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_SBCs, s_rec_l, 'all_to_all')

        nest.Connect(r_GBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_GBCs, s_rec_l, 'all_to_all')

        nest.Connect(r_MNTBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_MNTBCs, s_rec_l, 'all_to_all')

        nest.Connect(l_MSO, s_rec_r, 'all_to_all')

        #nest.Connect(l_MSO, s_rec_l, 'all_to_all')

        for r in range(len(spectro)-1):
            if(np.any(spectro[r][:] > 0)):
                nest.Connect(ANFs_noise, r_ANFs[10*r:10*(r+1)], 'all_to_all')
                nest.Connect(ANFs_noise, l_ANFs[10*r:10*(r+1)], 'all_to_all')

        for j in range(n_SBCs):
            nest.Connect(r_ANFs[j*ANFs2SBCs:j*ANFs2SBCs+ANFs2SBCs], r_SBCs[j], 'all_to_all', syn_spec = {"weight":ANFs2SBCs_weight})
            nest.Connect(l_ANFs[j*ANFs2SBCs:j*ANFs2SBCs+ANFs2SBCs], l_SBCs[j], 'all_to_all', syn_spec = {"weight":ANFs2SBCs_weight})

        for k in range(n_GBCs):
            nest.Connect(r_ANFs[k*ANFs2GBCs:k*ANFs2GBCs+ANFs2GBCs], r_GBCs[k], 'all_to_all', syn_spec = {"weight":ANFs2GBCs_weight})
            nest.Connect(l_ANFs[k*ANFs2GBCs:k*ANFs2GBCs+ANFs2GBCs], l_GBCs[k], 'all_to_all', syn_spec = {"weight":ANFs2GBCs_weight})

        for w in range(n_GBCs):
            for y in range(n_battery):
                nest.Connect(r_SBCs[w*int(ANFs2GBCs/ANFs2SBCs):w*int(ANFs2GBCs/ANFs2SBCs)+int(ANFs2GBCs/ANFs2SBCs)], l_MSO[w*n_battery + y], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight})
                nest.Connect(l_SBCs[w*int(ANFs2GBCs/ANFs2SBCs):w*int(ANFs2GBCs/ANFs2SBCs)+int(ANFs2GBCs/ANFs2SBCs)], l_MSO[w*n_battery + y], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight})    

        nest.Connect(r_GBCs, r_MNTBCs, 'one_to_one', syn_spec = {"weight":GBCs2MNTBCs_weight, "delay": delay})
        nest.Connect(l_GBCs, l_MNTBCs, 'one_to_one', syn_spec = {"weight":GBCs2MNTBCs_weight,  "delay": delay})

        for w in range(n_GBCs):
            for y in range(n_battery):
                nest.Connect(r_MNTBCs[w], l_MSO[w*n_battery + y], 'all_to_all', syn_spec = {"weight":MNTBCs2MSO_weights[y],  "delay": delay})

        mm = nest.Create('multimeter', 1, {'record_from': ['V_m']})
        nest.SetStatus(mm, {'interval': 0.01})
        nest.Connect(mm, l_MSO,'all_to_all')

        if(ild>=0): 
            mean_amplitude_r = mean_amplitude
            mean_amplitude_l = mean_amplitude/(10**(abs(ild)/20))
        else:
            mean_amplitude_l = mean_amplitude
            mean_amplitude_r = mean_amplitude/(10**(abs(ild)/20))

        def input_set_up(spectro, ms): 
            for r in range(0, len(spectro)-1):
                if spectro[r][ms] > 0:
                    r_ANFs_amp[10*r:10*(r+1)].set(amplitude = spectro[r][ms]*mean_amplitude_r)
                    l_ANFs_amp[10*r:10*(r+1)].set(amplitude = spectro[r][ms]*mean_amplitude_l)

        for i in range(sim_time):
            input_set_up(spectro,i)
            nest.Simulate(1)

        data_r = s_rec_r.get('events')
        #data_l = s_rec_l.get('events')
        #data_mso = mm.get('events')

        id_r_ANF1 = r_ANFs[0].get('global_id')
        id_r_SBC1 = r_SBCs[0].get('global_id')
        id_r_GBC1 = r_GBCs[0].get('global_id')
        id_r_MNTBC1 = r_MNTBCs[0].get('global_id')
        id_l_MSO1 = l_MSO[0].get('global_id')

        for h in range(n_battery):
            if((86311+h) in np.unique(data_r['senders'])):
                rate_group3[h][num_itd] = (np.unique(data_r['senders'][np.where(data_r['senders']==86311+h)], return_counts= True)[1][0])/sim_time*1000
            else:
                rate_group3[h][num_itd] = 0
            if((86321+h) in np.unique(data_r['senders'])):
                rate_group4[h][num_itd] = (np.unique(data_r['senders'][np.where(data_r['senders']==86321+h)], return_counts= True)[1][0])/sim_time*1000
            else:
                rate_group4[h][num_itd] = 0
            if((86331+h) in np.unique(data_r['senders'])):
                rate_group5[h][num_itd] = (np.unique(data_r['senders'][np.where(data_r['senders']==86331+h)], return_counts= True)[1][0])/sim_time*1000
            else:
                rate_group5[h][num_itd] = 0


        np.savetxt('left_rate_group3_itd_{}'.format(round(itd,2)), rate_group3[:,num_itd])
        np.savetxt('left_rate_group4_itd_{}'.format(round(itd,2)), rate_group4[:,num_itd])
        np.savetxt('left_rate_group5_itd_{}'.format(round(itd,2)), rate_group5[:,num_itd])
        
    #np.savetxt('rates_group3', rate_group3)
    #np.savetxt('rates_group4', rate_group4)
    #np.savetxt('rates_group5', rate_group5)

In [10]:
#itds = np.arange(0.85,1,0.05)
#ilds = np.linspace(1.7,2,len(itds))
itds = [0.25]
ilds = [0.5]
print(itds)
print(ilds)

[0.25]
[0.5]


In [11]:
MNTBCs2MSO_weights = [-2.0, -4.0, -6.0, -8.0, -10.0, -12.0, -14.0, -16.0, -20.0, -24.0]
n_battery = len(MNTBCs2MSO_weights)

In [12]:
rate_group3 = np.zeros([n_battery, len(itds)])
rate_group4 = np.zeros([n_battery, len(itds)])
rate_group5 = np.zeros([n_battery, len(itds)])
simulate(itds, ilds, MNTBCs2MSO_weights, 1000, rate_group3, rate_group4, rate_group5)

0.25
0.5

Apr 02 14:19:47 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.01 ms.

Apr 02 14:20:39 NodeManager::prepare_nodes [Info]: 
    Preparing 1232064 nodes for simulation.

Apr 02 14:20:39 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 1232064
    Simulation time (ms): 1
    Number of OpenMP threads: 16
    Number of MPI processes: 1

Apr 02 14:20:39 SimulationManager::run [Info]: 
    Simulation finished.

Apr 02 14:20:40 NodeManager::prepare_nodes [Info]: 
    Preparing 1232064 nodes for simulation.

Apr 02 14:20:40 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 1232064
    Simulation time (ms): 1
    Number of OpenMP threads: 16
    Number of MPI processes: 1

Apr 02 14:20:40 SimulationManager::run [Info]: 
    Simulation finished.

Apr 02 14:20:40 NodeManager::prepare_nodes [Info]: 
    Preparing 1232064 nodes for simulation.

Apr 02 14:20:40 SimulationManager::start_updating_ [Info]: 
    Nu